<a href="https://colab.research.google.com/github/spe301/AI-generated-AI/blob/main/DataCollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Potosnail==0.0.9

In [2]:
import pandas as pd
import numpy as np
from google.colab import files
import zipfile
import io
from potosnail import MachineLearning, DeepLearning, DataHelper, Evaluater, Algorithms, Wrappers
from sklearn.datasets import load_breast_cancer, load_boston

#class instantiations
ml = MachineLearning()
dl = DeepLearning()
dh = DataHelper()
ev = Evaluater()
al = Algorithms()
wr = Wrappers()

def BigDawg(X, y, params, task, ts=150, epochs=50, batch_size=32, patience=5, regression=False):
  '''facilitates the Data Collection Process'''
  dl = DeepLearning()
  wr = Wrappers()
  if task == 'NLP':
    #if regression==False:
    X2, y2, _ = wr.TCP(X, y)
    #else:
      #X2, y2, _ = wr.TRP(X, y)
    func = dl.RNN 
  if task == 'CV':
    X2, _, y2, _ = dl.ModelReadyPixles(X, y, target_size=(ts, ts))
    func = dl.CNN
  if task == 'TC':
    X2 = np.array(X)
    y2 = dl.MulticlassOutput(np.array(y))
    func = dl.DeepTabularClassification
  if task == 'TR':
    X2 = np.array(X)
    y2 = np.array(y)
    func = dl.DeepTabularRegression
  try:
    df = dl.CollectPerformance(params, func, X2, y2, epochs=epochs, batch_size=batch_size, patience=patience, regression=regression)
  except:
    df = dl.CollectPerformance(params, func, X2, y2.reshape(-1), epochs=epochs, batch_size=batch_size, patience=patience, regression=regression)
  size = []
  n_features = []
  batch = []
  depth = []
  dominances = []
  for i in range(len(df)):
    size.append(X2.shape[0])
    n_features.append(X2.shape[1])
    dominances.append(max(pd.DataFrame(y2).value_counts())/len(y2))
    if task == 'CV':
      depth.append(X2.shape[3])
  df['len_dataset'] = size
  df['n_features'] = n_features
  if regression == False:
    df['dominant_class'] = dominances
  if task == 'CV':
    df['thickness'] = depth
  return df

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
#Dataframes
Wikipedia = pd.read_csv('https://raw.githubusercontent.com/spe301/AI-generated-AI/main/Data/Wikipedia.csv') #text
Twitter = pd.read_csv('https://raw.githubusercontent.com/spe301/AI-generated-AI/main/Data/Twitter.csv') #text

#Series
articles = Wikipedia['Text']
labels = Wikipedia['AI']
tweets = Twitter['tweet']
sentiments = Twitter['is_there_an_emotion_directed_at_a_brand_or_product']

#arrays
cancer_data = load_breast_cancer()['data']
diagnosis = load_breast_cancer()['target']
housing_data = load_boston()['data']
prices = load_boston()['target']

#images
#uploaded = files.upload()
#data = zipfile.ZipFile(io.BytesIO(uploaded['IMGDS1.zip']), 'r')
#data.extractall()

In [17]:
X = articles
y = labels

df = Wikipedia
data_str = 'Text'
output = len(np.unique(y))
embedding = dh.GetVocab(df, data_str)
rnn_grid = {'output_dim': [output], 'embedding': [embedding], 'nodes': [64], 'activation': ['relu'], 'regularizer': ['L1', None, 'L2'], 
            'stacking': [False, True], 'dropout': [False, True], 'optimizer': ['adam', 'rmsprop', 'sgd'], 'method': ['GRU', 'LSTM'], 'bidirectional': [True]}

#output = len(y)
#cnn_grid = {'output_dim': [output], 'base_filters':[32], 'kernel_size': [2], 'activation': ['relu'], 'nblocks': [3], 
            #'pool':[2], 'dropout': [False], 'closer': [False], 'optimizer': ['adam'], 'metrics': ['accuracy']}

#output = len(np.unique(y))
#tc_grid = {'output_dim': [output], 'nodes': [64, 128], 'activation': ['relu'], 'regularizer': [None], 'stacking': [False], 
           #'dropout': [False], 'nlayers': [4], 'closer': [False], 'loss_func': ['binary_crossentropy'], 'optimizer': ['adam']}

#tr_grid = {'nodes': [32, 64, 128], 'activation': ['relu'], 'regularizer': [None], 'stacking': [False], 'dropout': [False], 
           #'nlayers': [4], 'closer': [False], 'loss_func': ['MSE', 'MAE'], 'optimizer': ['rmsprop'], 'y_col': ['string']}

df5 = BigDawg(X, y, rnn_grid, 'NLP')
df5.shape

preforming 72 total fits ...
Epoch 1/50
preforming 72 total fits ...
Epoch 1/50
9/9 [==============================] - 6s 320ms/step - loss: 0.6923 - accuracy: 0.5078 - val_loss: 0.6950 - val_accuracy: 0.5000
Epoch 2/50
9/9 [==============================] - 1s 155ms/step - loss: 0.6805 - accuracy: 0.5703 - val_loss: 0.6975 - val_accuracy: 0.5000
Epoch 3/50
9/9 [==============================] - 1s 155ms/step - loss: 0.6590 - accuracy: 0.7124 - val_loss: 0.7077 - val_accuracy: 0.5000
Epoch 4/50
9/9 [==============================] - 1s 154ms/step - loss: 0.5726 - accuracy: 0.7847 - val_loss: 0.7442 - val_accuracy: 0.5000
Epoch 5/50
9/9 [==============================] - 1s 156ms/step - loss: 0.3705 - accuracy: 0.9470 - val_loss: 0.8952 - val_accuracy: 0.4583
Epoch 6/50
9/9 [==============================] - 1s 154ms/step - loss: 0.1387 - accuracy: 0.9908 - val_loss: 1.3975 - val_accuracy: 0.4583
1 of 72 fits complete!
Epoch 1/50
9/9 [==============================] - 5s 256ms/step - lo

KeyboardInterrupt: ignored

In [ ]:
df0 = pd.read_csv('https://raw.githubusercontent.com/spe301/AI-generated-AI/main/Data/NLP3.csv').drop(['Unnamed: 0', 'Unnamed: 0.1'], axis='columns')

results = pd.concat([df0, df3, df4])

In [ ]:
results.to_csv('drive/MyDrive/NLP3.csv')

# Image dataset directory graveyard
copy and paste these into a code cell when working with the dataset

Dogs and Cats
train_dir = r'dataset/training_set'

test_dir = r'dataset/test_set'

train_dogs = r'dataset/training_set/dogs'

train_cats = r'dataset/training_set/cats'

test_dogs = r'dataset/test_set/dogs'

test_cats = r'dataset/test_set/cats'

directories = [train_dir, test_dir, train_dogs, train_cats, test_dogs, test_cats]

classes = ['cats', 'dogs']